In [1]:
from os import listdir
from os.path import isfile, join

In [2]:
path_to_dataset = "../dataset"
dataset_files = [
    join(path_to_dataset, path) 
    for path in listdir(path_to_dataset) 
    if isfile(join(path_to_dataset, path))
]

print( dataset_files )

['../dataset/smaller_wiki_dump.txt', '../dataset/wiki_dump.tokenized.txt']


In [3]:
from gensim.parsing.preprocessing import preprocess_string
from pymorphy2.utils import word_splits
from os.path import getsize
from math import floor
import pymorphy2
import re

class DatasetReader:
    def __init__(self, datasetPath):
        self._datasetPath = datasetPath
        self._currentText = ""
        self._morph = pymorphy2.MorphAnalyzer(lang='uk')
        self._ukrLetters = re.compile("^[абвгґдеєжзиіїйклмнопрстуфхцчшщьюя]*$", re.IGNORECASE)
    
    def __iter__(self):
        fileSize = getsize( self._datasetPath )
        prevProgress = 0
        
        with open(self._datasetPath, "r") as file:
            line = file.readline()
            while line:
                progress = floor(100 * 2.0 * file.tell() / fileSize)
                if prevProgress != progress:
                    prevProgress = progress
                    print(progress)
                
                self._currentText += line
                if self._tryPopSentence():
                    yield self._currentSentence
                
                try:
                    line = file.readline()
                except:
                    line = None
    
    def _tryPopSentence(self):
        sentenceEnd = self._currentText.find('.')
        if( sentenceEnd == -1 ):
            return False
        
        words = preprocess_string(self._currentText[:sentenceEnd:].replace( '\'', '' ))
        ukrWords = list(filter(lambda x: self._ukrLetters.search(x) , words))
        
        self._currentText = self._currentText[sentenceEnd+1::]
        self._currentSentence = self.to_normal_form(ukrWords)
        return True
        
    def to_normal_form(self, words_list):
        if isinstance(words_list, str):
            words_list = [words_list, ]

        res = [
            self._morph.parse(word)[0].normal_form
            for word in words_list
        ]

        if len(res) == 1:
            res = res[0]

        return res
        

In [4]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

In [5]:
from gensim.models.callbacks import CallbackAny2Vec

class LossLogger(CallbackAny2Vec):
    '''Output loss at each epoch'''
    def __init__(self):
        self.epoch = 1
        self.prevLoss = 0
        self.losses = []

    def on_epoch_begin(self, model):
        print(f'Epoch: {self.epoch}', end='\t')

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        self.losses.append(loss - self.prevLoss)
        print(f'  Loss: {loss - self.prevLoss}')
        self.epoch += 1
        self.prevLoss = loss

In [6]:

dataset = DatasetReader( dataset_files[0] )
sentences = [s for s in dataset]


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200


In [ ]:
loss_logger = LossLogger()
model = Word2Vec(sentences=sentences, epochs=100, compute_loss=True, vector_size=100, window=5, min_count=1, workers=4, callbacks=[loss_logger])

Epoch: 1	  Loss: 11485060.0
Epoch: 2	  Loss: 7671484.0
Epoch: 3	  Loss: 6617866.0
Epoch: 4	  Loss: 6489120.0
Epoch: 5	  Loss: 4537418.0
Epoch: 6	  Loss: 4088584.0
Epoch: 7	  Loss: 4109736.0
Epoch: 8	  Loss: 4092760.0
Epoch: 9	  Loss: 4063252.0
Epoch: 10	  Loss: 4030312.0
Epoch: 11	  Loss: 4036164.0
Epoch: 12	  Loss: 4007992.0
Epoch: 13	  Loss: 2517252.0
Epoch: 14	  Loss: 1192376.0
Epoch: 15	  Loss: 1203256.0
Epoch: 16	  Loss: 1187208.0
Epoch: 17	  Loss: 1205328.0
Epoch: 18	  Loss: 1195736.0
Epoch: 19	  Loss: 1186632.0
Epoch: 20	  Loss: 1195920.0
Epoch: 21	  Loss: 1184224.0
Epoch: 22	  Loss: 1180296.0
Epoch: 23	  Loss: 1187192.0
Epoch: 24	  Loss: 1176224.0
Epoch: 25	  Loss: 1181960.0
Epoch: 26	  Loss: 1188232.0
Epoch: 27	  Loss: 1178712.0
Epoch: 28	  Loss: 1177104.0
Epoch: 29	  Loss: 1162632.0
Epoch: 30	  Loss: 1155448.0
Epoch: 31	  Loss: 1167608.0
Epoch: 32	  Loss: 1152488.0
Epoch: 33	  Loss: 1157608.0
Epoch: 34	  Loss: 1135400.0
Epoch: 35	  Loss: 1136976.0
Epoch: 36	  Loss: 1121416.0


In [ ]:
model.save("500mb_wiki.model")

In [ ]:
model.wv.most_similar("ялинка")
